In [30]:
import glob
import json
import pandas as pd

In [31]:
tweets_dict = {}

# Read all *.txt files in the ./tweet_jsons/ directory and print every tweet out
for filepath in glob.iglob('./tweet_jsons/*.json'):
    f = open(filepath, "r")
    tweets_df = pd.read_json(filepath, lines=True)
    tweets_dict[filepath[14:-5]] = tweets_df
    
print(tweets_dict.keys())

dict_keys(['pmarca', 'novogratz', 'rogerkver', 'needacoin', 'crypto_rand', 'ToneVays', 'legendofcrypto', 'CryptoChoe', 'cryptoBully', 'VitalikButerin', 'peterlbrandt', 'NicTrades', 'DiaryofaMademan', 'crypto_God', 'tuurdemeester', 'APompliano', 'lopp', 'nickszabo4', 'CryptoCobain', '6BillionPeople', 'cryptodemedici', 'CryptoOrca', 'VinnyLingham', 'WhalePanda', 'cryptopathic', 'IAMJOSEPHYYOUNG', 'CryptoHustle', 'gavinandresen', 'Crypto_goat_', 'PhilakoneCrypt', 'aantonop', 'brian_armstrong', 'cryptobanger', '22loops', 'CryptoYoda1338', 'VentureCoinist', 'secretsofcrypto', 'barrysilbert', 'Thecryptodog', 'onemanatatime', 'cryptomocho', 'officialmcafee', 'erikvoorhees', 'IOTA_Charles', 'thecryptomofo', 'cryptogat', 'balajis', 'ZeusZissou', 'CremeDeLaCrypto', 'CryptoTutor', 'cz_binance', 'jimmysong', 'justinsuntron', 'angelabtc', 'notsofast', 'charlieshrem', 'SatoshiLite'])
